In [1]:
import pandas as pd
import tmap as tmap
from faerun import Faerun

from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors
from matplotlib.colors import ListedColormap
from rdkit.Chem.Descriptors import ExactMolWt

import pandas as pd
import numpy as np

In [2]:
#Train test data
train_test_path = "/home/mylab-pharma/Code/tuele/XO/data/train_test_data/XO_train_test_data_for_tmap.xlsx"
train_dataset = pd.read_excel(train_test_path, sheet_name='train_dataset')
test_dataset = pd.read_excel(train_test_path, sheet_name='test_dataset')
validation_dataset = pd.read_excel(train_test_path, sheet_name='validation_dataset')

#Add data belonging
belong_col_name = "Belong_to"
train_dataset[belong_col_name] = "Training data"
test_dataset[belong_col_name] = "Testing data"
validation_dataset[belong_col_name] = "Validation data"

#Train_test_dataset = combine of all three
all_dataset = pd.concat([train_dataset, validation_dataset, test_dataset], axis=0)

#Getting the classifier
all_data_path = "/home/mylab-pharma/Code/tuele/XO/data/raw_data/20240530_data_XO_with substructure.xlsx"

zgb_classifer_df = pd.read_excel(all_data_path, sheet_name='Substructures')

In [3]:
print(len(train_dataset), len(test_dataset), len(validation_dataset), len(all_dataset))

337 73 73 483


In [4]:
all_dataset.head()

,CID,SMILES,IC50(nM),aid,Type,Substructure,Belong_to
0,145967694,CC1=CC2=C(C=C1)N=C(O2)/C(=N/O)/CC3=CC=CC=C3,17500.0,1389558,active,16,Training data
1,76329670,CC1(C=CC2=CC(=C(C=C2O1)O)C(=O)/C=C/C3=CC(=C(C=...,1800.0,1485273,inactive,6,Training data
2,5320686,C1=CC(=CC=C1/C=C/C(=O)OC[C@@H]2[C@H]([C@@H]([C...,100000.0,399340,active,10,Training data
3,155903284,C1=CC(=CC=C1C2=NC=NN2)NC(=O)C3C(NC(=O)NC3=O)O,1400.0,1806026,inactive,1,Training data
4,137648214,CCCCC1=NN2C(=N1)C3=C(NC2=O)NN=C3,529.0,1485284,inactive,1,Training data


In [5]:
zgb_dict = zgb_classifer_df.set_index('Code')['Substructure'].to_dict()
zgb_dict

{1: '1,2,4-Triazole',
 2: 'catechol',
 3: 'pyrimidine',
 4: '1H-pyrazole-4-carbonitrile',
 5: '4-methylthiazole-5-carboxylic acid',
 6: 'butein',
 7: 'coumarin',
 8: 'isocytosine',
 9: 'fisetin',
 10: 'kaempferol',
 11: 'luteolin',
 12: 'diosmetin',
 13: 'quercetin',
 14: 'taxifolin',
 15: 'myricetin',
 16: 'khác'}

In [6]:
#MACCS
from tqdm import tqdm

def tmap_maccs_fpts(data):
    Maccs_fpts = []
    count = 0
    with tqdm(total=len(data), desc='Progress') as pbar:
        for i in data:
            try:
                mol = Chem.MolFromSmiles(i)
            except:
                print("An exception occurred with " + str(count))
                continue
            fpts = MACCSkeys.GenMACCSKeys(mol)
            mfpts = np.array(fpts)
            mfpts = tmap.VectorUint(mfpts)
            Maccs_fpts.append(mfpts)
            count += 1
            pbar.update(1)  # Update the progress bar
    return np.array(Maccs_fpts)

#Morgan2
def tmap_morgan_fpts(data):
    Morgan_fpts = []
    count = 0
    with tqdm(total=len(data), desc='Progress') as pbar:
        for i in data:
            try:
                mol = Chem.MolFromSmiles(i)
            except:
                print("An exception occurred with " + str(count))
                continue
            fpts = AllChem.GetMorganFingerprintAsBitVect(mol, 2, 1024)
            mfpts = np.array(fpts)
            mfpts = tmap.VectorUint(mfpts)
            Morgan_fpts.append(mfpts)
            count += 1
            pbar.update(1)  # Update the progress bar
    return Morgan_fpts

In [39]:
fps = tmap_morgan_fpts(all_dataset["SMILES"])

Progress: 100%|██████████| 483/483 [00:00<00:00, 2181.51it/s]


In [40]:
len(fps[0])

1024

# Generate Tmap Layout

Generate labels

In [41]:
ic50 = []
labels = []
belong_to_groups = []
zbg_classifier_groups = []
active_inactive_groups = []
molecular_weight = []
chembl_url = "https://www.ebi.ac.uk/chembl/compound_report_card/"
pubchem_url = "https://pubchem.ncbi.nlm.nih.gov/compound/"

for i, row in all_dataset.iterrows():
    smiles = row['SMILES']
    mol = AllChem.MolFromSmiles(smiles)
    cid = str(row['CID'])
    if cid[-1].isalpha():
        cid = cid[:-1]
    
    labels.append(
            f'{smiles}__<a href="{pubchem_url+str(cid)}" target="_blank">{smiles}</a>__{smiles}'.replace(
                "'", ""
            )
        )
    ic50.append(row['IC50(nM)'])
    # zbg labels groups
    zbg_classifier_groups.append(zgb_dict[row['Substructure']])
    #Active inactive label groups
    active_inactive_groups.append(row['Type'])
    #Belong to groups
    belong_to_groups.append(row[belong_col_name])
    #Molecular weight
    smiles_mw = ExactMolWt(mol)
    molecular_weight.append(smiles_mw)

In [42]:
print(len(belong_to_groups), len(ic50), len(zbg_classifier_groups), len(active_inactive_groups), len(labels))

483 483 483 483 483


__Preprocessing the groups labels__

In [43]:
import scipy.stats as ss
# Create the labels and the integer encoded array for the groups,
# as they're categorical
zgb_labels_groups, zgb_groups = Faerun.create_categories(zbg_classifier_groups)
activity_labels_groups, activity_groups = Faerun.create_categories(active_inactive_groups)
belong_labels_groups, belong_groups = Faerun.create_categories(belong_to_groups)
#scale IC50
ic50_ranked = ss.rankdata(np.array(ic50) / max(ic50)) / len(ic50)
mw_ranked = ss.rankdata(np.array(molecular_weight) / max(molecular_weight)) / len(molecular_weight)

In [44]:
belong_labels_groups

[(0, 'Testing data'), (1, 'Training data'), (2, 'Validation data')]

In [45]:

bits = len(fps[0])
k = 300
enc = tmap.Minhash(bits)
lf = tmap.LSHForest(bits, 128)
lf.batch_add(enc.batch_from_binary_array(fps))
lf.index()
cfg = tmap.LayoutConfiguration()
cfg.k = k
cfg.sl_repeats = 2
cfg.mmm_repeats = 2
cfg.node_size = 2
x, y, s, t, _ = tmap.layout_from_lsh_forest(lf, config=cfg)

In [46]:
len(x)

483

In [47]:
from matplotlib.colors import LinearSegmentedColormap
faerun = Faerun(view="front", clear_color="#ffffff", coords=False)

custom_cmap = ListedColormap(
["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", 
 "#9467bd", "#8c564b", "#e377c2", "#7f7f7f",
 "#17becf", "#000080", "#ff1493", "#00ff00", 
 "#ffa500", "#008080", "#ff00ff", "#00ffff"],
name="custom",)

faerun.add_scatter(
        "chembl",
        {"x": x, 
         "y": y, 
         "c": [zgb_groups, belong_groups, activity_groups, ic50_ranked, mw_ranked], 
         "labels": labels},
        colormap=[custom_cmap, custom_cmap, custom_cmap, "viridis", "viridis"],
        # change the size of view point
        # point_scale=4.5,
        # max_point_size=10,
        point_scale=9,
        max_point_size=20,
        has_legend=True,
        categorical=[True, True, True, False, False],
        shader="smoothCircle",
        legend_labels=[zgb_labels_groups, belong_labels_groups, activity_labels_groups],
        selected_labels=["SMILES", "PubChem CID", "Name"],
        series_title=["ZGB Classifier group", "Dataset label group", "Activity label group", "IC50 (nM)", "Molecular Weight"],
        max_legend_label=[None, None, None, str(round(max(ic50))), str(round(max(molecular_weight)))],
        min_legend_label=[None, None, None, str(round(min(ic50))), str(round(min(molecular_weight)))],
        title_index=2,
        legend_title=""
    )
faerun.add_tree(
    "pubchem_tree", {"from": s, "to": t}, point_helper="chembl", color="#222222"
)
# You may see a different Tmap from what us generated since the index of the data-points are randomly generated depending on the runtime environment.
# Howerver, the way your tmap branches connected will be similar to ours.
faerun.plot(file_name="../../results/tmap/final_XO_ECFP4_tmap", template="smiles")
# LOOK OUT!!! BECAUSE THE LEGEND BACKGROUND IS BLACK, YOU SHOULD CHANGE IT IN HTML FILE, WHICH IN LEGEND -> background color 

/home/mylab-pharma/Code/tuele/XO/results/tmap/final_XO_ECFP4_tmap.html